In [0]:
patients_df=spark.read.csv('/Volumes/workspace/synthea/synthea_datasets/patients.csv',header=True,inferSchema=True)


patients_df.write.format('delta').mode('overwrite').save('/Volumes/workspace/synthea/synthea_datasets/patients_delta')

In [0]:
# Read from Delta instead
patients_df = spark.read.format("delta").load(
    '/Volumes/workspace/synthea/synthea_datasets/patients_delta'
)

# Display sample
display(patients_df)
#patients_df.printSchema()

In [0]:
from pyspark.sql.functions import col, when
deceased_df=patients_df.withColumn("status",
                          when(col("deathdate").isNull(),
                               "alive").otherwise("deceased"))
display(deceased_df)

In [0]:
marital_df=deceased_df.withColumn("marital_status",
                          when(col("marital").isNull(),
                               "unknown").otherwise(col("marital")))
display(marital_df)
#marital_df.write.format('delta').mode('overwrite').save('/Volumes/workspace/synthea/synthea_datasets/marital_delta')

In [0]:
df=marital_df.toDF(*[c.lower() for c in marital_df.columns])
display(df)

In [0]:
from pyspark.sql import functions as F

patients_silver_df = df.select(
    "id",
    "birthdate",
    F.when(F.col("deathdate").isNull(), F.lit("9999-12-31")).otherwise(F.col("deathdate")).alias("deathdate"),
    "gender",
    "race",
    "city",
    "state",
    "county",
    "fips",
    "healthcare_expenses",
    "healthcare_coverage",
    "income",
    "status",
    "marital_status",
    "ethnicity"
)



display(patients_silver_df)

In [0]:
patients_silver_df.write.format('delta').mode('overwrite').save('/Volumes/workspace/synthea/synthea_datasets/patients_silver_delta')